In [1]:
# Mengimpor modul utilitas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# **MEMASUKKAN KODE UCI REPO UNTUK MENGAMBIL DATA WINE QUALITY**
from ucimlrepo import fetch_ucirepo

# fetch dataset (Wine Quality ID=186)
wine_quality = fetch_ucirepo(id=186)

# data (as pandas dataframes)
# X adalah fitur (variabel independen)
# y adalah target (variabel dependen, yaitu 'quality')
X = wine_quality.data.features
y = wine_quality.data.targets

# Mengimpor model pembelajaran mesin untuk prediksi
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression

# **MENGGANTI BAGIAN PEMUATAN DATA DF DAN PEMISAHAN TRAIN/TARGET**
# Data train (X) dan target (y) sudah diambil dari UCI di atas.
# Kita perlu mengonversi 'y' menjadi Series atau DataFrame dengan nama kolom 'quality'
# agar dapat diakses dengan kunci: y_series = y['quality']

# Memisahkan antara data pelatihan ke dalam himpunan data pelatihan dan validasi
# Menggunakan X (features) dan y['quality'] (target)
X_train, X_test, y_train, y_test = train_test_split(
    X, y['quality'], test_size=0.20, random_state=42
)

# Menginisialisasi semua objek model dengan parameter default
model_1 = LinearRegression()
model_2 = xgb.XGBRegressor()
model_3 = RandomForestRegressor()

# melatih semua model pada himpunan data pelatihan
# **CATATAN: Variabel y_target pada kode asli Anda TIDAK terdefinisi,
# sudah diganti dengan y_train yang benar.**
model_1.fit(X_train, y_train)
model_2.fit(X_train, y_train)
model_3.fit(X_train, y_train)

# memprediksi output pada himpunan data validasi
pred_1 = model_1.predict(X_test)
pred_2 = model_2.predict(X_test)
pred_3 = model_3.predict(X_test)

# prediksi akhir setelah rata-rata prediksi dari semua 3 model
pred_final = (pred_1 + pred_2 + pred_3) / 3.0

# mencetak kesalahan kuadrat rata-rata antara nilai riil dan nilai prediksi
print(f"Mean Squared Error (MSE) dari Ensemble (Averaging): {mean_squared_error(y_test, pred_final)}")

# **INFORMASI DATASET TAMBAHAN (Opsional)**
print("\n--- Metadata Dataset Wine Quality ---")
print(wine_quality.metadata)
print("\n--- Informasi Variabel Dataset Wine Quality ---")
print(wine_quality.variables)

Mean Squared Error (MSE) dari Ensemble (Averaging): 0.38908347298224133

--- Metadata Dataset Wine Quality ---
{'uci_id': 186, 'name': 'Wine Quality', 'repository_url': 'https://archive.ics.uci.edu/dataset/186/wine+quality', 'data_url': 'https://archive.ics.uci.edu/static/public/186/data.csv', 'abstract': 'Two datasets are included, related to red and white vinho verde wine samples, from the north of Portugal. The goal is to model wine quality based on physicochemical tests (see [Cortez et al., 2009], http://www3.dsi.uminho.pt/pcortez/wine/).', 'area': 'Business', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 4898, 'num_features': 11, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['quality'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Wed Nov 15 2023', 'dataset_doi': '10.24432/C56S3T', 'creators': ['Paulo Cortez', 'A. Cerdeira', 'F. Alm

In [2]:
# Mengimpor modul utilitas
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

# **MEMASUKKAN KODE UCI REPO UNTUK MENGAMBIL DATA ADULT (KLASIFIKASI)**
from ucimlrepo import fetch_ucirepo

# fetch dataset (Adult ID=2)
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
# X adalah fitur (variabel independen)
# y adalah target (variabel dependen, yaitu income: >50K, <=50K)
X = adult.data.features
y = adult.data.targets

# Mengimpor model pembelajaran mesin untuk prediksi
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
# Menambahkan kelas untuk Logistik Regression agar bisa mengatasi konvergensi
from sklearn.linear_model import LogisticRegression
# mengimpor pengklasifikasi suara
from sklearn.ensemble import VotingClassifier



X = pd.get_dummies(X, drop_first=True)


y_series = y.iloc[:, 0].str.strip().str.replace('.', '', regex=False)


# **MENGGANTI BAGIAN PEMUATAN DATA DF DAN PEMISAHAN TRAIN/TARGET**
# Memisahkan antara data pelatihan ke dalam himpunan data pelatihan dan validasi
X_train, X_test, y_train, y_test = train_test_split(
    X, y_series, test_size=0.20, random_state=42
)


# Menginisialisasi semua objek model dengan parameter default
# Menambahkan max_iter dan solver untuk membantu konvergensi Logistic Regression
model_1 = LogisticRegression(solver='liblinear', random_state=42) 
model_2 = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_3 = RandomForestClassifier(random_state=42)

# Membuat model akhir menggunakan pengklasifikasi pemungutan suara
final_model = VotingClassifier(
    estimators=[('lr', model_1), ('xgb', model_2), ('rf', model_3)],
    voting='hard'
)

# melatih semua model pada himpunan data kereta
# CATATAN: Karena data sudah di-pre-process dan dipisah, ini siap dilatih.
final_model.fit(X_train, y_train)

# memprediksi output pada himpunan data pengujian
pred_final = final_model.predict(X_test)


# Menggunakan Akurasi sebagai metrik untuk VotingClassifier 'hard'
accuracy = final_model.score(X_test, y_test)
print(f"Accuracy dari Ensemble (Max Voting 'hard'): {accuracy}")

# Jika Anda ingin menggunakan log_loss, ganti voting='hard' menjadi voting='soft'
# dan gunakan pred_proba. Contoh:
"""
final_model_soft = VotingClassifier(
    estimators=[('lr', model_1), ('xgb', model_2), ('rf', model_3)],
    voting='soft'
)
final_model_soft.fit(X_train, y_train)
pred_proba_final = final_model_soft.predict_proba(X_test)
print(f"Log Loss dari Ensemble (Max Voting 'soft'): {log_loss(y_test, pred_proba_final)}")
"""

# **INFORMASI DATASET TAMBAHAN (Opsional)**
print("\n--- Metadata Dataset Adult ---")
print(adult.metadata)
print("\n--- Informasi Variabel Dataset Adult ---")
print(adult.variables)

C:\Users\ACER\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [18:23:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy dari Ensemble (Max Voting 'hard'): 0.8647763332992118

--- Metadata Dataset Adult ---
{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Tue Sep 24 2024', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': "Extraction was done by Barry Becke

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.ensemble import StackingRegressor
from sklearn.datasets import make_regression # Import untuk membuat data sintetis

# --- MEMUAT DATA SINTETIS (make_regression) sebagai pengganti data UCI ---
# Membuat dataset regresi dengan 1000 sampel dan 10 fitur
X_data, y_data = make_regression(n_samples=1000, n_features=10, 
                                 n_informative=5, random_state=42)

X_clean = pd.DataFrame(X_data)
y_clean = pd.Series(y_data)

# Memisahkan antara data pelatihan ke dalam himpunan data pelatihan dan validasi
X_train, X_test, y_train, y_test = train_test_split(
    X_clean, y_clean, test_size=0.20, random_state=42
)

# --- MODEL IMPLEMENTASI STACKING DENGAN SKLEARN (Pengganti vecstack) ---
# menginisialisasi semua objek model dasar (estimators)
model_1 = LinearRegression()
# model_2: Diganti dari XGBRegressor menjadi KNeighborsRegressor karena XGBoost tidak tersedia
model_2 = KNeighborsRegressor() 
model_3 = RandomForestRegressor(random_state=42)

# menempatkan semua objek model dasar dalam format list of tuples (name, estimator)
estimators = [
    ('lr', model_1),
    ('knn', model_2),
    ('rf', model_3)
]

# Inisialisasi StackingRegressor (final_estimator adalah meta-model)
final_model = StackingRegressor(
    estimators=estimators, 
    final_estimator=LinearRegression(), # Meta-model (level-two model)
    cv=4 # Cross-validation folds (setara dengan n_folds di vecstack)
)

# melatih model Stacking pada himpunan data pelatihan
final_model.fit(X_train, y_train)

# memprediksi output akhir
pred_final = final_model.predict(X_test)

# mencetak kesalahan kuadrat rata-rata antara nilai riil dan nilai prediksi
print(f"Mean Squared Error (MSE) dari Ensemble Stacking: {mean_squared_error(y_test, pred_final):.4f}")

Mean Squared Error (MSE) dari Ensemble Stacking: 0.0000


In [4]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from ucimlrepo import fetch_ucirepo

# --- MEMUAT DATA DARI UCI (WINE QUALITY, ID=186) ---
wine_quality = fetch_ucirepo(id=186)
X = wine_quality.data.features
y = wine_quality.data.targets['quality'] # Target Regresi

# Tentukan rasio pembagian untuk Train/Validation/Test [cite: 143, 144, 145]
train_ratio = 0.70
validation_ratio = 0.20
test_ratio = 0.10

# 1. Melakukan pemisahan Train/Rest (Test + Validation)
x_train, x_rest, y_train, y_rest = train_test_split(
    X, y, test_size=1 - train_ratio, random_state=42
)

# 2. Melakukan pemisahan Validation/Test dari data 'Rest'
# Perhitungan test_size: test_ratio / (test_ratio + validation_ratio)
x_val, x_test, y_val, y_test = train_test_split(
    x_rest, y_rest, test_size=test_ratio / (test_ratio + validation_ratio), random_state=42
)

# menginisialisasi semua objek model dasar dengan parameter default [cite: 154]
model_1 = LinearRegression()
model_2 = xgb.XGBRegressor(random_state=42)
model_3 = RandomForestRegressor(random_state=42)

# Melatih dan memprediksi
# Pelatihan Model Pertama (Linear Regression)
model_1.fit(x_train, y_train)
val_pred_1 = model_1.predict(x_val) # Prediksi pada Validation
test_pred_1 = model_1.predict(x_test) # Prediksi pada Test

# Pelatihan Model Kedua (XGBoost)
model_2.fit(x_train, y_train)
val_pred_2 = model_2.predict(x_val)
test_pred_2 = model_2.predict(x_test)

# Pelatihan Model Ketiga (Random Forest)
model_3.fit(x_train, y_train)
val_pred_3 = model_3.predict(x_val)
test_pred_3 = model_3.predict(x_test)

# Mengonversi hasil prediksi ke Dataframe untuk digabungkan
val_preds = pd.DataFrame({'pred_1': val_pred_1, 'pred_2': val_pred_2, 'pred_3': val_pred_3}, index=x_val.index)
test_preds = pd.DataFrame({'pred_1': test_pred_1, 'pred_2': test_pred_2, 'pred_3': test_pred_3}, index=x_test.index)

# kumpulan data validasi gabungan bersama dengan semua data validasi yang diprediksi (fitur meta) [cite: 174, 175]
df_val = pd.concat([x_val.reset_index(drop=True), val_preds.reset_index(drop=True)], axis=1)
df_test = pd.concat([x_test.reset_index(drop=True), test_preds.reset_index(drop=True)], axis=1)
y_val_blend = y_val.reset_index(drop=True)
y_test_blend = y_test.reset_index(drop=True)

# Membuat model akhir menggunakan fitur meta (Meta Model) [cite: 177, 178]
final_model = LinearRegression()
final_model.fit(df_val, y_val_blend)

# mendapatkan hasil akhir [cite: 180, 181]
final_pred = final_model.predict(df_test)

# printing kesalahan kuadrat rata-rata antara nilai riil dan nilai prediksi [cite: 182]
print("--- Metode Blending (Regresi) ---")
print(f"Mean Squared Error (MSE): {mean_squared_error(y_test_blend, final_pred):.4f}")

--- Metode Blending (Regresi) ---
Mean Squared Error (MSE): 0.3621


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor # Pengganti XGBoost
from sklearn.datasets import make_regression 

# --- MEMUAT DATA SINTETIS (make_regression) sebagai pengganti data UCI ---
# Membuat dataset regresi dengan 1000 sampel dan 10 fitur
X_data, y_data = make_regression(n_samples=1000, n_features=10, 
                                 n_informative=5, random_state=42)

X = pd.DataFrame(X_data)
y = pd.Series(y_data)

# Memisahkan antara data pelatihan ke dalam himpunan data pelatihan dan validasi
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# --- KOREKSI ERROR: GANTI 'base_estimator' menjadi 'estimator' ---
# menginisialisasi model pengantongan menggunakan KNeighborsRegressor sebagai model dasar
model = BaggingRegressor(
    estimator=KNeighborsRegressor(), # Parameter telah diganti menjadi 'estimator'
    random_state=42
)

# model pelatihan
model.fit(X_train, y_train)

# memprediksi output pada himpunan data pengujian
pred_final = model.predict(X_test)

# mencetak kesalahan kuadrat rata-rata antara nilai riil dan nilai prediksi
print("--- Metode Bagging (Regresi) ---")
print(f"Mean Squared Error (MSE): {mean_squared_error(y_test, pred_final):.4f}")

--- Metode Bagging (Regresi) ---
Mean Squared Error (MSE): 898.7090


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from ucimlrepo import fetch_ucirepo

# --- MEMUAT DATA DARI UCI (WINE QUALITY, ID=186) ---
wine_quality = fetch_ucirepo(id=186)
X = wine_quality.data.features
y = wine_quality.data.targets['quality'] # Target Regresi

# Memisahkan antara data pelatihan ke dalam himpunan data pelatihan dan validasi [cite: 241, 242]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# menginisialisasi modul boosting (Gradient Boosting Regressor) [cite: 243, 244]
model = GradientBoostingRegressor(random_state=42)

# melatih model pada himpunan data kereta [cite: 246]
model.fit(X_train, y_train)

# memprediksi output pada himpunan data pengujian [cite: 248, 249]
pred_final = model.predict(X_test)

# mencetak kesalahan kuadrat rata-rata antara nilai riil dan nilai prediksi [cite: 250]
print("--- Metode Boosting (Regresi) ---")
print(f"Mean Squared Error (MSE): {mean_squared_error(y_test, pred_final):.4f}")

--- Metode Boosting (Regresi) ---
Mean Squared Error (MSE): 0.4616
